## Исследование надёжности заёмщиков

Нужно провести исследование и ответить на вопрос: "Влияет ли семейное положение, доход и количество детей клиента на факт погашения кредита в срок?" 

### План работы:

1. [Знакомство с данными](#1.-Знакомство-с-данными.) 


2. [Предобработка данных](#2.-Предобработка-данных.)
  - [Обработка пропусков](#Обработка-пропусков)
  - [Замена типа данных](#Замена-типа-данных)
  - [Обработка дубликатов](#Обработка-дубликатов)
  - [Лемматизация](#Лемматизация)
  - [Категоризация данных](#Категоризация-данных)  
  
 
3. [Исследование о наличии зависимости между возвратом кредита в срок и:](#Возможные-факторы,-влияющие-на-возврат-кредита-в-срок.)
  - [наличием детей](#Наличие-детей)
  - [семейным положением](#Семейное-положение)
  - [уровнем дохода](#Уровень-дохода) 
  - [целью кредита](#Цель-кредита)  
  
  
4. [Общий вывод](#4.-Общий-вывод.)


### Описание данных
Входные данные от банка: информация о клиентах, об их семейном положении, количестве детей, образовании, работе, цели получении кредита и был ли он погашен в срок.


---

### 1. Знакомство с данными. 

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/Users/olga/SynologyDrive/Yandex_data_scientist/yandex-praktikum-projects/yandex_practicum/datasets/credit_scoring/data.csv')
display(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Файл открыт и изучен. Имеется таблица с 21525 записями. В столбцах "days_employed" и "total_income" одинаковое количество пропусков, нужно проверить, совпадают ли эти строки. Столбцы "days_employed", "total_income" имеют тип float64, нужно будет привести к типу int64. При выводе первых 10 записей, обнаруживаются проблемы со значениями: в столбце "days_employed" есть отрицательные или аномально высокие значения. Отрицательные значения могли появиться при вычете из даты начала работы даты окончания. В столбце 'education' встречаются написания в разном регистре. Категориальные переменные: 'education', 'education_id', 'family_status', 'family_status_id, 'gender', 'income_type', 'purpose'; количественные: 'children', 'days_employed', 'dob_years', 'total_income'; логические: 'debt'. 

### 2. Предобработка данных.

### Обработка пропусков

In [3]:
# Проверим, совпадают ли строки, в которых пропущены значения 'days_employed' и 'total_income'
days_nan = df['days_employed'].isnull()
income_nan = df['total_income'].isnull()
if len(df[days_nan & income_nan]) == days_nan.sum():
    print("Строки с пропусками 'days_employed' и 'total_income' совпадают, их количество {}, {:.0%} от общего числа записей.".format(days_nan.sum(), days_nan.sum()/len(df)))

Строки с пропусками 'days_employed' и 'total_income' совпадают, их количество 2174, 10% от общего числа записей.


In [4]:
# проведем группировку по типу занятости и заменим пропущенные значения в 'total income' средним в каждой группе 
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda row: row.fillna(row.mean()))
df['total_income'].isnull().sum()

0

In [5]:
# проведем группировку по возрасту и заменим пропущенные значения в 'days_employed' средним для каждого возраста
df['days_employed'] = df.groupby('dob_years')['days_employed'].apply(lambda row: row.fillna(row.mean()))
df['days_employed'].isnull().sum()

0

In [6]:
df.groupby('income_type')['days_employed'].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
безработный,2.0,366413.652744,40855.478519,337524.466835,351969.059790,366413.652744,380858.245699,395302.838654
в декрете,1.0,-3296.759962,NaN,-3296.759962,-3296.759962,-3296.759962,-3296.759962,-3296.759962
госслужащий,1459.0,808.430236,26854.666177,-15193.032201,-4454.221934,-2385.358043,-925.502949,291629.371951
компаньон,5085.0,1185.971443,22127.929394,-17615.563266,-2672.323106,-1342.646018,-527.447471,337795.125514
пенсионер,3856.0,348236.188645,59828.602594,-1026.405485,340199.573133,360505.668544,380751.301140,401755.400475
предприниматель,2.0,92435.152603,131459.636874,-520.848083,45957.152260,92435.152603,138913.152946,185391.153289
сотрудник,11119.0,1065.904462,24012.456276,-18388.949901,-2895.108262,-1403.779132,-575.345846,337795.125514
студент,1.0,-578.751554,NaN,-578.751554,-578.751554,-578.751554,-578.751554,-578.751554


In [7]:
# изучим все столбцы в таблице
# столбец 'children'
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [8]:
# есть явно ошибочные данные '-1' и '20' детей, посмотрим на распределение значений в этом столбце:
df['children'].describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [9]:
# преобладают значения '0', сумма строк со значениями '-1' и '20' составляют меньше процента от суммы строк с '0', 
# можем заменить ошибочные значения на '0':
df.loc[df['children'] == 20, 'children'] = 0
df.loc[df['children'] == -1, 'children'] = 0
df['children'].value_counts()


0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [10]:
# 'days_employed'
df['days_employed'].describe()

count     21525.000000
mean      63311.243751
std      137241.206602
min      -18388.949901
25%       -2518.168900
50%       -1022.586185
75%        -115.370479
max      401755.400475
Name: days_employed, dtype: float64

In [11]:
# в столбце очень много отрицательных значений, по всей видимости из-за особенностей вычисления стажа работы, и есть аномально большие значения
# избавимся от отрицательных значений:
df.loc[:, 'days_employed'] = df.loc[:, 'days_employed'].abs()
df['days_employed'].describe()

count     21525.000000
mean      66833.934767
std      135560.562991
min          21.296136
25%         946.323545
50%        2274.043065
75%        6667.385826
max      401755.400475
Name: days_employed, dtype: float64

In [12]:
# столбец 'dob_years'
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [13]:
# количество строк, где возраст равен нулю
len(df[df['dob_years']==0])

101

In [14]:
# количество "нулевых возрастов" составляет меньше процента от общего количества, можем заменить на медиану в срезе образования
df['dob_years'] = df.groupby('education_id')['dob_years'].apply(lambda row: row.replace(to_replace=0, value=row.mean()))
df['dob_years'].describe()
                                                               

count    21525.000000
mean        43.495149
std         12.219127
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [15]:
# столбец 'education'
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [16]:
# приведем все значения к одному регистру:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [17]:
# столбец 'family_status'
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [18]:
# столбец 'gender'
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [19]:
# исследуем строку с неправильным значением пола
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [20]:
df.groupby('gender')['total_income'].describe()
# по размеру заработка не можем сказать М это или Ж

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,14236.0,155274.529940,85879.720160,20667.263793,100004.600727,138954.735939,186950.903220,1.715018e+06
M,7288.0,191068.190580,114275.033763,21205.280566,127915.577626,164755.475659,223080.616346,2.265604e+06
XNA,1.0,203905.157261,NaN,203905.157261,203905.157261,203905.157261,203905.157261,2.039052e+05


In [21]:
df[df['income_type']=='компаньон']['gender'].describe()

count     5085
unique       3
top          F
freq      3197
Name: gender, dtype: object

In [22]:
df[df['family_status']=='гражданский брак']['gender'].describe()

count     4177
unique       3
top          F
freq      2868
Name: gender, dtype: object

In [23]:
df[df['education_id']==2.0]['gender'].describe()

count     744
unique      3
top         F
freq      460
Name: gender, dtype: object

In [24]:
# судя по должности, семейному статусу и образованию, скорей всего пропущенное значение - Ж
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [25]:
# 'income_type'
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [26]:
# 'debt'
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [27]:
# 'total_income'
df['total_income'].describe()

count    2.152500e+04
mean     1.673959e+05
std      9.790695e+04
min      2.066726e+04
25%      1.077982e+05
50%      1.519313e+05
75%      2.024175e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [28]:
# 'purpose'
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Проведена обработка пропусков в данных. Были найдены строки, в которых были одновременно пропущены значения стажа и дохода, что логично - если человек не работал, не было и дохода. Таких строк было 10% от общего количества записей. Пропущенные значения были типа NaN, для замены на средние значения, в зависимости от возраста и типа деятельности, использовался метод fillna. 

Были изучены все столбцы таблицы:
'children' - были найдены ошибочные значения '-1' и '20', они составили меньше процента от общего количества и были заменены на нули;

'days_employed' - много отрицательных значений, возможно из-за вычитания даты окончания работы из даты ее начала. Аномально высокие значения появились из-за пропущенных или неправильных тысячелетий?

'dob_years' - строки с нулевым возрастом (возможно клиенты просто не пожелали его указывать) были заменены на медиану;

'education' - все строки были приведены к строчному регистру для выявления дубликатов;

Во всех оставшихся столбцах все в порядке, кроме столбца 'gender', где есть один пропуск, возможно человек намеренно не захотел указывать. Проанализтровав другоие столбы этой записи, пришли к выводу, что значение можно заменить на 'F'. 

### Замена типа данных

In [29]:
# замена типа 'days_employed', 'total_income', 'dob_years' с float на int. 'dob_years' стал float после замены нулевых значений срединм
df['days_employed'] = df['days_employed'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df['total_income'] = df['total_income'].astype('int')


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Была проведена замена типа столбца 'days_employed', 'total_income', 'dob_years' с float на int с помощью метода astype, так как значения уже и так численные, и метод to_numeric здесь не подходит.

### Обработка дубликатов

In [31]:
df.duplicated().sum()

71

In [32]:
# были найдены 71 дубликат, удаляем их и проверяем
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Был найден 71 дубликат. Дубликаты могли появиться из-за ошибок загрузки данных. Это небольшое количество (<1%), они были удалены с помощью drop_duplicates.

### Лемматизация

In [33]:
m = Mystem()
df['purpose_lemma'] = df['purpose'].apply(m.lemmatize)
df.head()
df['purpose_lemma'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

Была проведена лемматизация с помощью библиотеки pymystem3. Из результата видно, что цели кредита можно выделить в 4 группы: автомобиль, свадьба, недвижимость и образование.

### Категоризация данных

In [34]:
# пишем функцию для разделения целей кредита на 4 категории
def purpose_group(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'свадьба' in purpose:
        return 'свадьба'
    elif ('недвижимость' in purpose) or ('жилье' in purpose):
        return 'недвижимость'
    else:
        return 'образование'
df['purpose_category'] = df['purpose_lemma'].apply(purpose_group)
df['purpose_category'].value_counts()


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

In [35]:
# для проверки влияния дохода на возврат кредита, нужно категоризировать клиентов по уровню дохода. 
# Выберем интервалы разделения из квартилей:
df['total_income'].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [36]:
def income_group(income):
    if income <= 107623:
        return 'низкий'
    elif income <= 151887:
        return 'средний'
    elif income <= 202417:
        return 'высокий'
    else:
        return 'очень высокий'
df['total_income_category'] = df['total_income'].apply(income_group)
df['total_income_category'].value_counts()    

высокий          5818
низкий           5364
средний          5363
очень высокий    4909
Name: total_income_category, dtype: int64

In [37]:
# для определения влияния наличия детей, выделим 2 категории:
def children_yes_no(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
df['children_yes_no'] = df['children'].apply(children_yes_no)
df['children_yes_no'].value_counts() 


нет детей    14214
есть дети     7240
Name: children_yes_no, dtype: int64

In [38]:
def children_category(children):
    if children == 0:
        return 'без детей'
    elif children <=2:
        return 'малодетная'
    else:
        return 'многодетная'
df['children_category'] = df['children'].apply(children_category)
df['children_category'].value_counts() 

без детей      14214
малодетная      6860
многодетная      380
Name: children_category, dtype: int64

### Вывод

Категоризация данных была проведена по целям кредита, по уровню дохода клиентов и есть ли у клиентов дети, по количеству детей.

### Возможные факторы, влияющие на возврат кредита в срок.

### Наличие детей

In [39]:
pd.options.display.float_format = '{:,.1f}%'.format
children_effect = pd.pivot_table(df, index=['children_yes_no'], columns='debt', values='education_id', aggfunc='count')#.sort_values(by = 'debt')
children_effect['debt_procent'] = children_effect[1] / (children_effect[0] + children_effect[1]) * 100
children_effect

debt,0,1,debt_procent
children_yes_no,,,
есть дети,6571,669,9.2%
нет детей,13142,1072,7.5%


In [40]:
# посмотрим, как влияет количество детей
children_num_effect = pd.pivot_table(df, index=['children_category'], columns='debt', values='education_id', aggfunc='count')
children_num_effect['debt_procent'] = children_num_effect[1] / (children_num_effect[0] + children_num_effect[1]) * 100
children_num_effect.sort_values(by='debt_procent', ascending=False)

debt,0,1,debt_procent
children_category,,,
малодетная,6222,638,9.3%
многодетная,349,31,8.2%
без детей,13142,1072,7.5%


### Вывод

Среди тех, у кого есть дети, больше должников. Наименее надежными, согласно статистике, являются семьи с 1-2 детьми.

### Cемейное положение

In [41]:
family_effect = pd.pivot_table(df,index=['family_status'], columns='debt', values='education_id', aggfunc='count')
family_effect['debt_procent'] = family_effect[1] / (family_effect[0] + family_effect[1]) * 100
family_effect.sort_values(by = 'debt_procent', ascending=False)

debt,0,1,debt_procent
family_status,,,
Не женат / не замужем,2536,274,9.8%
гражданский брак,3763,388,9.3%
женат / замужем,11408,931,7.5%
в разводе,1110,85,7.1%
вдовец / вдова,896,63,6.6%


In [42]:
family_effect_gender = pd.pivot_table(df,index=['family_status', 'gender'], columns='debt', values='education_id', aggfunc='count')
family_effect_gender['debt_procent'] = family_effect_gender[1] / (family_effect_gender[0] + family_effect_gender[1]) * 100
family_effect_gender.sort_values(by = 'debt_procent', ascending=False)

,debt,0,1,debt_procent
family_status,gender,,,
вдовец / вдова,M,44,11,20.0%
Не женат / не замужем,M,925,156,14.4%
гражданский брак,M,1150,155,11.9%
в разводе,M,235,24,9.3%
женат / замужем,M,4178,401,8.8%
гражданский брак,F,2613,233,8.2%
женат / замужем,F,7230,530,6.8%
Не женат / не замужем,F,1611,118,6.8%
в разводе,F,875,61,6.5%


### Вывод

Те, кто не женат / не замужем и состоят в гражданском браке чаще не возвращают кредит в срок по сравнению с другими клиентами. При группировке по полу, оказалось, что мужчины чаще не возвращают кредит в срок. Самыми злостными неплательщиками являются мужчины-вдовцы, а женщины-вдовы, напротив, являются самыми надежными в данной выборке.

### Уровень дохода

In [43]:
income_effect = pd.pivot_table(df,index=['total_income_category'], columns='debt', values='education_id', aggfunc='count')
income_effect['debt_procent'] = income_effect[1] / (income_effect[0] + income_effect[1]) * 100
income_effect.sort_values(by = 'debt_procent', ascending=False)

debt,0,1,debt_procent
total_income_category,,,
средний,4888,475,8.9%
высокий,5324,494,8.5%
низкий,4937,427,8.0%
очень высокий,4564,345,7.0%


### Вывод

Люди со средним и высоким доходом чаще становятся должниками, возможно они больше любят рисковать :). 

- Как разные цели кредита влияют на его возврат в срок?

In [44]:
#pd.pivot_table(df,index=['purpose_category'], values=["debt"]).sort_values(by = 'debt')
purpose_effect = pd.pivot_table(df,index=['purpose_category'], columns='debt', values='education_id', aggfunc='count')
purpose_effect['debt_procent'] = purpose_effect[1] / (purpose_effect[0] + purpose_effect[1]) * 100
purpose_effect.sort_values(by = 'debt_procent', ascending=False)

debt,0,1,debt_procent
purpose_category,,,
автомобиль,3903,403,9.4%
образование,3643,370,9.2%
свадьба,2138,186,8.0%
недвижимость,10029,782,7.2%


In [45]:
purpose_effect_income = pd.pivot_table(df,index=['purpose_category', 'total_income_category'], columns='debt', values='education_id', aggfunc='count')
purpose_effect_income['debt_procent'] = purpose_effect_income[1] / (purpose_effect_income[0] + purpose_effect_income[1]) * 100
purpose_effect_income.sort_values(by = 'debt_procent', ascending=False)

debt                                       0    1  debt_procent
purpose_category total_income_category                         
образование      средний                 901  101         10.1%
                 высокий                 989  108          9.8%
автомобиль       средний                1024  111          9.8%
                 низкий                  943  101          9.7%
                 высокий                1009  108          9.7%
свадьба          средний                 523   53          9.2%
                 низкий                  548   54          9.0%
образование      низкий                  951   91          8.7%
свадьба          высокий                 567   53          8.5%
автомобиль       очень высокий           927   83          8.2%
образование      очень высокий           802   70          8.0%
недвижимость     средний                2440  210          7.9%
                 высокий                2759  225          7.5%
                 низкий                 2495  181          6.8%
                 очень высокий          2335  166          6.6%
свадьба          очень высокий           500   26          4.9%

### Вывод

Клиенты, взявшие кредит на недвижимость или свадьбу, чаще возвращают деньги в срок, чем клиенты, потратившие их на образование или автомобиль. При группировке по уровню дохода, можно видеть, что должниками чаще становятся люди со средним и высоким уровнем дохода, взявшие кредит на учебу.

### 4. Общий вывод.

Анализ показал, что клиенты с детьми чаще не возвращают кредит в срок. Согласно статистике, семьи с 1-2 детьми самые ненадежные. Те, кто не состоят или не состояли в официальных отношениях чаще не возвращают кредит в срок по сравнению с другими клиентами. При рассмотрении зависимости от пола, оказывается, что самыми злостными неплательщиками являются мужчины-вдовцы, а женщины-вдовы, напротив, являются самыми надежными клиентами. Люди со средним и высоким доходом чаще становятся должниками. Клиенты, взявшие кредит на недвижимость чаще возвращают кредит в срок, возможно из-за сделок с недвижимостью или сдачей в аренду. "Свадебные" кредиты также чаще возвращаются в срок, скорей всего из-за щедрости гостей на свадьбе, приводящей к ее "окупаемости". А вот люди со средним и высоким уровнем дохода, взявшие кредит на учебу, наименее надежная категория.